In [8]:
def exception_vung_tau(list_text):
    if 'BA RIA' in list_text and "VUNG TAU" in list_text:
        list_text.reverse()
        index_vung_tau = list_text.index('VUNG TAU')
        index_ba_ria = list_text.index('BA RIA')
        if index_vung_tau<index_ba_ria:       
            list_text.remove('BA RIA')
            list_text.remove('VUNG TAU')
            list_text.insert(0,'BA RIA VUNG TAU')
        list_text.reverse()
    return list_text
    
def exception_ha_noi(list_text):
    if list_text.count('HA NOI')>1:
        list_text.remove('HA NOI')
    return list_text
    

In [9]:
from cleanText import text_clean, clearColumns
from inputData import textTransfer
from standardAddress import addressList, loadAddressList, normalizeAddress
from validAddress import ValidAddress
import pandas as pd
matchAddress = ValidAddress()

provinceName = "./data/provinceLib.csv"

standardAddressList = [text_clean(item) for item in addressList(provinceName)]
listAddress = loadAddressList(provinceName)

# replace the fail characteristic and the unexpected location words
def replaceCharList(text):
    replaceList = [".,", "-,", ",,","--",".-",'-CU-','-VIET NAM NINH','VIET NAM NINH','-VIET NAM CAI','VIET NAM CAI',"-VIET NAM NOI","VIET NAM NOI","-VIET NAM DA",",VIET NAM",",VN", ",VIETNAM",", VIET NAM",", VN", ", VIETNAM",'-VIETNAM','-VIET NAM',"VIET NAM","VN", "VIETNAM",]
    textResults = text_clean(text)
    for item in replaceList:
        textResults = textResults.replace(item," ")
    return textResults
    
def parseAddress(item):
    if (item.find(',') != -1):
        textList = item.split(",")
    elif (item.find("-") != -1):
        textList = item.split("-")
    else:
        textList = [0]
    if textList[0] ==0 and len(textList)==1:
        textList = [0]
    else:
        textList = [clearColumns(value.strip()) for value in textList if clearColumns(value.strip()) != '']
    textList = exception_vung_tau(textList)
    textList= exception_ha_noi(textList)
    extractList = ["0","0","0"]
    lenList = len(textList)
    
    missedTextList = None
    if lenList < 3:
        missedItem = textTransfer(item)
        missedCharList = [",,", ', ,', '-']
        for char in missedCharList:
            missedItem = missedItem.replace(char, ",")
        
        missedTextList = missedItem.split(",")
        lenListTemp = len(missedTextList)
        if lenListTemp < 3:
            extractList = ["0", "0", "0"]
        else:
            extractList = [missedTextList[lenListTemp-1],
                            missedTextList[lenListTemp-2], missedTextList[lenListTemp-3]]
            for value in extractList:
                missedTextList.remove(value)
        extractList = normalizeAddress(extractList)
        return ','.join(missedTextList), extractList[2] + ", "+ extractList[1]+ ", " + extractList[0]
    else:
        
        extractList = [textList[lenList-1],
                        textList[lenList-2], textList[lenList-3]]
        for value in extractList:
            textList.remove(value)
        extractList = normalizeAddress(extractList)
        return ','.join(textList), extractList[2] + ", "+ extractList[1] + ", " + extractList[0]
        

In [10]:
line = "BINH HOA 2 TAN PHUOC KHANH TAN UYEN BINH DUONG"
address = line.strip()
results = replaceCharList(line)
print(results)
# kiem tra
other, results = parseAddress(results)
result = matchAddress.valid_address(results)
print(results)
print('result ',result)
print('other ',other)

BINH HOA 2 TAN PHUOC KHANH TAN UYEN BINH DUONG
TAN UYEN , B, INH DUONG
result  {'province': None, 'district': None, 'ward': None}
other  INH ,HOA ,2 ,TAN PHUOC KHANH ,B


In [11]:
import pandas as pd

In [12]:
def check_dia_chi(file_path,ouput_name):
    list_full = []
    list_other = []
    addresses = []
    df = pd.read_excel(file_path)
    IDs = df['MAKH']
    lines = df['DIACHI_BK']
    for line in lines:
        address = line.strip()
        addresses.append(address)
        
        results = replaceCharList(line)
        
        # kiem tra
        other, results = parseAddress(results)
        result = matchAddress.valid_address(results)
        if result['province'] is not None and result['district'] is not None and result['ward'] is not None:
            list_full.append(True)
        else:
            list_full.append(False)
        if len(other.strip())>0:
            list_other.append(True)
        else:
            list_other.append(False)
    df = pd.DataFrame(list(zip(IDs,addresses, list_full,list_other)),
                columns =['MAKH','DIACHI_BK', 'Dia chi dung 3 cap do','Dia chi duoi cap xa'])
    df.to_excel(ouput_name)

In [13]:
file_path = 'datatest.xlsx'
output = 'ketquadiachi.xlsx'
check_dia_chi(file_path,output)